In [21]:
import numpy as np

In [4]:
import numpy as np
import networkx as nx
from trap import create_trap_graph

# 1) Load your logical (no‐time) gate layers
with open("time_steps_raw_fixedphi.txt", "r") as file:
    gt_no_time = [eval(line.strip()) for line in file]

# 2) Build the trap graph
G = create_trap_graph()
interaction_sites = {n for n,d in G.nodes(data=True) if d['type']=="interaction"}
standard_sites    = {n for n,d in G.nodes(data=True) if d['type']=="standard"}

L, Q = len(gt_no_time), 8

# 3) Initial positions (storage/home)
pos_hist = np.zeros((1, Q, 2), dtype=int)
pos_hist[0] = np.array([
    (0,1),(0,3),(1,0),(1,4),
    (3,0),(3,4),(4,1),(4,3)
])
home_pos = {q: tuple(pos_hist[0, q]) for q in range(Q)}

# 4) Output container
scheduled_ops = []

def append_timestep(ops):
    """Record gate ops for one tick, carry forward positions."""
    global pos_hist
    scheduled_ops.append(ops)
    if pos_hist.shape[0] < len(scheduled_ops) + 1:
        pos_hist = np.vstack([pos_hist, pos_hist[-1:]])

def plan_and_move(q, goal):
    """
    Move qubit q from its current node to `goal`,
    one hop per tick, avoiding other ions.
    """
    global pos_hist
    start = tuple(pos_hist[-1, q])
    if start == goal:
        return

    occupied = {tuple(p) for p in pos_hist[-1]}
    occupied.discard(start)
    occupied.discard(goal)

    def w(u, v, data):
        return 1e6 if (u in occupied or v in occupied) else 1

    path = nx.dijkstra_path(G, source=start, target=goal, weight=w)
    for hop_node in path[1:]:
        append_timestep([])  # movement‐only tick
        newpos = pos_hist[-1].copy()
        newpos[q] = hop_node
        pos_hist = np.vstack([pos_hist, newpos[np.newaxis]])

# 5) Scheduler loop
for layer in range(L):
    ops = gt_no_time[layer]

    # compute next‐MS qubits (union over all MS in next layer)
    if layer + 1 < L:
        next_ms_gates = [g for g in gt_no_time[layer+1] if g[0]=='MS']
        next_ms_qubits = set(q for g in next_ms_gates for q in g[2])
    else:
        next_ms_qubits = set()

    # --- Two‐qubit MS gates (now all of them) ---
    ms_gates = [g for g in ops if g[0]=='MS']
    for _, angle, (q1, q2) in ms_gates:
        # pick best interaction site
        currs = [tuple(pos_hist[-1, q]) for q in (q1, q2)]
        sites = np.array(list(interaction_sites))
        dists = [sum(abs(s - np.array(c)).sum() for c in currs) for s in sites]
        target = tuple(sites[int(np.argmin(dists))])

        # shuttle in both qubits
        plan_and_move(q1, target)
        plan_and_move(q2, target)

        # perform MS gate (sort qubits so never inverted)
        pair = tuple(sorted((q1, q2)))
        append_timestep([('MS', angle, pair)])

        # return those not reused in the next MS
        to_return = {q1, q2} - next_ms_qubits
        for q in to_return:
            plan_and_move(q, home_pos[q])

    # --- Single‐qubit rotations ---
    sq_ops = [(name, angle, q) for name, angle, q in ops if name in ('RX', 'RY')]
    if sq_ops:
        # if any still in interaction zone, move them out
        for _, _, q in sq_ops:
            curr = tuple(pos_hist[-1, q])
            if curr in interaction_sites:
                occupied = {tuple(p) for p in pos_hist[-1]}
                occupied.discard(curr)
                for nb in G.neighbors(curr):
                    if nb in standard_sites and nb not in occupied:
                        plan_and_move(q, nb)
                        break

        # apply all single‐qubit gates in one tick
        append_timestep(sq_ops)

        # after, return those not used in the next MS
        for _, _, q in sq_ops:
            if q not in next_ms_qubits:
                plan_and_move(q, home_pos[q])

    # --- Idle qubits movement ---
    involved = set()
    involved |= {q for _,_,g in ms_gates for q in g}
    for _, _, q in sq_ops:
        involved.add(q)

    for q in range(Q):
        if q not in involved and tuple(pos_hist[-1, q]) != home_pos[q]:
            plan_and_move(q, home_pos[q])

# 6) Final cleanup: ensure all home
for q in range(Q):
    plan_and_move(q, home_pos[q])

# Output results
print(f"Total time steps: {len(scheduled_ops)}")
print("scheduled_ops =")
for ops in scheduled_ops:
    print(ops)


Total time steps: 611
scheduled_ops =
[('RY', 1.5707963267948966, 6), ('RX', 0.6154797086703869, 7)]
[('RX', 1.5707963267948966, 6), ('RY', 1.570796326794897, 7)]
[('RX', -1.5707963267948966, 7)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (6, 7))]
[]
[]
[]
[]
[('RY', 3.1415926535897927, 6), ('RX', -1.5707963267948966, 7)]
[('RY', 0.7853981633974495, 7)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (6, 7))]
[]
[]
[]
[]
[('RX', 0.7853981633974492, 6), ('RY', 1.5707963267948963, 7)]
[('RY', 1.5707963267948963, 6), ('RX', 0.562617536425785, 7)]
[]
[]
[('MS', 1.5707963267948966, (5, 7))]
[]
[]
[('RX', -2.356194490192345, 6)]
[('RY', 0.39269908169872403, 5), ('RX', -1.5707963267948966, 7)]
[]
[]
[('MS', 1.5707963267948966, (5, 7))]
[]
[]
[('RX', -1.5707963267948966, 5), ('RX', -1.3744467859455334, 7)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (4, 7))]
[]
[]
[]
[]
[('RY', -0.39269908169872414, 5)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (5, 6))]
[]
[]
[]
[]
[('RY', 0.19634954084936218, 4), ('RX', -1.57

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio # type: ignore
import os

# ─── CONFIG ────────────────────────────────────────────────────────────────
# Replace this with your real pos_hist if it's not already in scope:
# pos_hist = YOUR_POS_HIST_ARRAY  
# ────────────────────────────────────────────────────────────────────────────

T, Q, _ = pos_hist.shape

frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Determine grid extents
rows = int(pos_hist[:,:,0].max() + 1)
cols = int(pos_hist[:,:,1].max() + 1)

frame_files = []
for t in range(T):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.grid(True)

    for q in range(Q):
        r, c = pos_hist[t, q]
        ax.scatter(c, r, s=200, edgecolor='black')
        ax.text(c, r, str(q), color='white',
                ha='center', va='center', fontsize=10, weight='bold')

    ax.set_title(f"Time step {t}")
    ax.set_aspect('equal')
    filename = os.path.join(frames_dir, f"frame_{t:03d}.png")
    fig.savefig(filename, dpi=100)
    plt.close(fig)
    frame_files.append(filename)

# Build the GIF
gif_path = 'movement.gif'
with imageio.get_writer(gif_path, mode='I', duration=1) as writer:
    for ff in frame_files:
        writer.append_data(imageio.imread(ff))

print(f"Saved animation to {gif_path}")


Total time steps: 701
scheduled_ops =
[('RX', -2.3561944901923453, 7)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (6, 7))]
[]
[]
[]
[]
[('RY', 0.7853981633974483, 6), ('RX', -1.5707963267948966, 7)]
[('RX', -1.5707963267948966, 6)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (6, 7))]
[]
[]
[]
[]
[('RY', -0.7853981633974482, 6), ('RX', -1.178097245096172, 7)]
[]
[]
[('MS', 1.5707963267948966, (5, 7))]
[]
[]
[('RX', 2.356194490192344, 6)]
[('RY', 0.39269908169872414, 5), ('RX', -1.5707963267948966, 7)]
[('RX', -1.5707963267948966, 5)]
[]
[]
[('MS', 1.5707963267948966, (5, 7))]
[]
[]
[('RY', -0.3926990816987241, 5), ('RX', -1.3744467859455343, 7)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (4, 7))]
[]
[]
[]
[]
[('RX', -1.5707963267948966, 5)]
[]
[]
[]
[]
[('MS', 1.5707963267948966, (5, 6))]
[]
[]
[]
[]
[('RY', 0.19634954084936207, 4), ('RX', -1.5707963267948966, 7)]
[('RY', 0.7853981633974483, 5), ('RX', -1.5707963267948966, 6), ('RX', -1.5707963267948966, 4)]
[]
[]
[]
[]
[('MS', 1.5707963267948966

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import os

# ─── Replace this with your real pos_hist if needed ─────────────────────────
# pos_hist = np.load("pos_hist.npy")  # or however you get it
# ────────────────────────────────────────────────────────────────────────────

# Make sure pos_hist is a (T, Q, 2) array of ints
T, Q, _ = pos_hist.shape

# Where to save the individual PNGs
out_dir = "history_pngs"
os.makedirs(out_dir, exist_ok=True)

# Determine trap grid size from the data
n_rows = int(pos_hist[:,:,0].max() + 1)
n_cols = int(pos_hist[:,:,1].max() + 1)

for t in range(T):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_title(f"Time step {t}")
    ax.set_xlim(-0.5, n_cols - 0.5)
    ax.set_ylim(n_rows - 0.5, -0.5)
    ax.set_xticks(range(n_cols))
    ax.set_yticks(range(n_rows))
    ax.grid(True)

    # Plot each qubit as a colored dot with its index
    for q in range(Q):
        r, c = pos_hist[t, q]
        ax.scatter(c, r, s=150, edgecolor='black', linewidth=1)
        ax.text(c, r, str(q),
                ha='center', va='center',
                color='white', fontsize=9, fontweight='bold')

    ax.set_aspect('equal')
    ax.invert_yaxis()  # so row 0 is at the top, if you prefer

    filename = os.path.join(out_dir, f"step_{t:03d}.png")
    fig.savefig(filename, dpi=100, bbox_inches='tight')
    plt.close(fig)

print(f"Wrote {T} PNG files to `{out_dir}`")


Wrote 577 PNG files to `history_pngs`


In [5]:
from verifier import verifier
from fidelity import fidelity
import trap

pos_sequence = [
    [tuple(pos) for pos in pos_hist[t+1]]
    for t in range(len(scheduled_ops))
]

print(pos_sequence)
print(scheduled_ops)

graph = trap.create_trap_graph()

verifier(pos_sequence, scheduled_ops, graph)
fidelity(pos_sequence, scheduled_ops, graph)

[[(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 2)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 2)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), 

0.922461936773054

In [48]:
with open('circuit_output.txt', 'w', encoding='utf-8') as f:
    for ops in scheduled_ops:
        # repr(ops) gives you something like:
        #   [('RX', -2.35619, 7), ...]
        f.write(repr(ops) + '\n')